In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from matplotlib import colors
#import xgboost as xgb
#
import os
import sys
#
from datetime import datetime, timedelta
from time import time
from uuid import uuid4
from scipy.ndimage import convolve1d
import numpy as np
import pandas as pd
import tensorflow as tf

#
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import cross_validate
# 
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

%matplotlib inline

In [120]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [6]:
!unzip "/content/drive/MyDrive/pai_machine_metric11.zip"

Archive:  /content/drive/MyDrive/pai_machine_metric11.zip
  inflating: pai_machine_metric.csv  


In [7]:
!unzip "/content/drive/MyDrive/pai_task_table11.zip"

Archive:  /content/drive/MyDrive/pai_task_table11.zip
  inflating: pai_task_table.csv      


In [214]:
train_data = pd.read_csv('/content/pai_machine_metric.csv')
train_data

,worker_name,machine,start_time,end_time,machine_cpu_iowait,machine_cpu_kernel,machine_cpu_usr,machine_gpu,machine_load_1,machine_net_receive,machine_num_worker,machine_cpu
0,8fa54171acbc9f16704c03f137ac053ede9a97432cfc94...,5248e29b2b7f2b5f8b974608,2134401,2134599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9b69319eb5d19b4ea2514fcb4d761c3d00be925b65f529...,5248e29b2b7f2b5f8b974608,2806218,2806331,0.006080,9.226264,40.158927,192.551088,89.731273,3.845883e+08,15.672816,49.391271
2,b37cd5d848323f23352464f71d3c27ef4796532ee09d59...,5248e29b2b7f2b5f8b974608,3084090,3084411,0.000699,10.140853,40.807522,150.497242,72.377214,7.027994e+08,14.234469,50.949074
3,8a3db37b3795c7c753963d0ef966df9703d39b9f918e1a...,5248e29b2b7f2b5f8b974608,2119980,2134354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,f114ac24745ec00da7107387afc99ed7cd8132530e8ef7...,5248e29b2b7f2b5f8b974608,2164803,2171278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
499994,adee9d2bd7265e9c6a3a9b599ac147f1ad9cc34e56f0d5...,274faea7c2b83356d5de3b77,3132103,3133131,0.004562,4.102616,17.383334,51.223102,21.956170,5.512585e+08,5.043801,21.490512
499995,fcf321bd4d26c893574e29d2e17740e3d61015b05a84f6...,274faea7c2b83356d5de3b77,3211881,3211982,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290
499996,bdc422a0fc9e4e9f47c4deea474db2bca79c1c626e369a...,274faea7c2b83356d5de3b77,3211882,3211973,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290
499997,32d57bdcdcca54e964f75fcba6c975f5b1eb5258727f81...,274faea7c2b83356d5de3b77,3244419,3244470,0.002670,1.787369,18.590296,21.676219,20.071826,5.256312e+07,5.345062,20.380335


In [215]:
train_data.columns

Index(['worker_name', 'machine', 'start_time', 'end_time',
       'machine_cpu_iowait', 'machine_cpu_kernel', 'machine_cpu_usr',
       'machine_gpu', 'machine_load_1', 'machine_net_receive',
       'machine_num_worker', 'machine_cpu'],
      dtype='object')

In [216]:
test_data = pd.read_csv('/content/pai_task_table.csv')
test_data

,job_name,task_name,inst_num,status,start_time,end_time,plan_cpu,plan_mem,plan_gpu,gpu_type
0,3e6f005d5fc08a0108f4eb1e,ps,1,Terminated,675282,675318,600.0,29.296875,NaN,NaN
1,aa93a6ed0b24235487081bf1,ps,1,Terminated,731739,731772,600.0,29.296875,NaN,NaN
2,96e053818c246cd68f301d40,ps,2,Terminated,591455,802150,200.0,4.882812,NaN,NaN
3,10e530eb422cec7bb19c6fd7,ps,1,Terminated,687588,807680,100.0,0.976562,NaN,NaN
4,2883edbad23c6bef16f655c0,ps,1,Terminated,610350,813276,100.0,0.976562,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
499994,76f391e1deed03eb87ade370,tensorflow,1,Terminated,4480035,4483303,600.0,29.296875,100.0,MISC
499995,64002d792fba760cddebd253,tensorflow,1,Terminated,4482654,4483307,600.0,29.296875,100.0,P100
499996,fe3990737fef541f2f4dc6dd,tensorflow,1,Terminated,4482651,4483320,600.0,29.296875,100.0,P100
499997,e162cfb7391cd08ff4bcc7e5,tensorflow,1,Terminated,4482936,4483322,600.0,29.296875,25.0,T4


In [217]:
train_data.dtypes

worker_name             object
machine                 object
start_time               int64
end_time                 int64
machine_cpu_iowait     float64
machine_cpu_kernel     float64
machine_cpu_usr        float64
machine_gpu            float64
machine_load_1         float64
machine_net_receive    float64
machine_num_worker     float64
machine_cpu            float64
dtype: object

In [218]:
test_data.dtypes

job_name       object
task_name      object
inst_num        int64
status         object
start_time      int64
end_time        int64
plan_cpu      float64
plan_mem      float64
plan_gpu      float64
gpu_type       object
dtype: object

In [126]:
len(train_data.columns)

12

In [127]:
len(test_data.columns)

10

In [219]:
train_data.isna().sum()

worker_name                0
machine                    0
start_time                 0
end_time                   0
machine_cpu_iowait     42111
machine_cpu_kernel     59422
machine_cpu_usr        43613
machine_gpu            53859
machine_load_1         50347
machine_net_receive    46597
machine_num_worker     52708
machine_cpu            60953
dtype: int64

In [220]:
test_data.isna().sum()

job_name          0
task_name         0
inst_num          0
status            0
start_time        0
end_time          0
plan_cpu        351
plan_mem        351
plan_gpu      99150
gpu_type      99009
dtype: int64

In [130]:
train_data["machine_gpu"].unique()

array([    nan, 192.551, 150.497, ...,  29.884,  21.676,  40.311])

In [131]:
# Get count duplicates When having nan values
df2 = train_data.pivot_table(columns=['machine_gpu'], aggfunc='size')
print(df2)

machine_gpu
0.000000      253
0.000788        1
0.001293        1
0.001471        1
0.001474        1
             ... 
753.967063      1
759.646033      1
759.985202      1
760.536408      1
762.246263      1
Length: 428565, dtype: int64


In [132]:
# Get count duplicates When having nan values
df2 = train_data.pivot_table(columns=['machine_cpu'], aggfunc='size')
print(df2)

machine_cpu
1.200159     1
1.205916     1
1.306423     1
1.314211     1
1.314386     1
            ..
99.134186    1
99.178589    1
99.219481    1
99.271547    1
99.272349    1
Length: 422824, dtype: int64


In [221]:
df2 = train_data.groupby(train_data.columns.tolist(), as_index=True).size()
print(df2.head(50))

worker_name                                                   machine                   start_time  end_time  machine_cpu_iowait  machine_cpu_kernel  machine_cpu_usr  machine_gpu  machine_load_1  machine_net_receive  machine_num_worker  machine_cpu
00003397282f303aa87d59bfbad5214bfaafdab3da31d9b197086937ca15  94f2961239dc5c855088c15f  4572964     4574676   0.002649            10.681014           36.110710        499.884615   58.278061       6.508965e+08         14.779004           46.794373      1
0000415c89b73cf16ad1dcecf959a774810c060167a1050344a861d1de60  40fd6adce4fc460cb21f5224  3367625     3382114   0.001817            16.048790           17.387714        371.630885   93.201314       6.499883e+08         9.769371            33.438322      1
000043b4624f0029dcab376fb056adef06374fded6930b75e925fb9861c6  405d86056a38bc086a130c73  5353067     5396008   0.001590            26.273453           42.201389        151.375958   277.574524      6.147367e+08         19.713717           68.476

In [222]:
tst2 = test_data.groupby(test_data.columns.tolist(), as_index=True).size()
print(tst2.head(50))

job_name                  task_name       inst_num  status      start_time  end_time  plan_cpu  plan_mem   plan_gpu  gpu_type
0000407f42f4dddf0a49d7e5  tensorflow      1         Terminated  2345202     2345247   600.0     29.296875  50.0      T4          1
0000694f3c7e9654d7cb59e1  worker          30        Terminated  1508103     1509101   400.0     14.648438  50.0      MISC        1
0000797b79842dd4521e97b3  PyTorchWorker   1         Terminated  1105020     1105089   800.0     15.625000  100.0     T4          1
00007e24e5fedde13a7cba0d  worker          50        Terminated  2432201     2432529   400.0     9.765625   50.0      MISC        1
0000c3df064b632f2008c101  worker          10        Terminated  2590612     2591047   50.0      48.828125  25.0      MISC        1
0000d08291c27948e15cbdc0  PyTorchWorker   1         Terminated  1144905     1145023   50.0      48.828125  300.0     V100        1
00013e1168e274915b7b2691  tensorflow      1         Terminated  1779678     1780167   60

In [135]:
# Use DataFrame.groupby() and nunique() to columns
df2 = (train_data['worker_name']).unique()
print(df2) 

['8fa54171acbc9f16704c03f137ac053ede9a97432cfc94e7d1d6893d9b5a'
 '9b69319eb5d19b4ea2514fcb4d761c3d00be925b65f52964197052de6c1e'
 'b37cd5d848323f23352464f71d3c27ef4796532ee09d5964b173744a908a' ...
 'bdc422a0fc9e4e9f47c4deea474db2bca79c1c626e369acab7ede4b6e794'
 '32d57bdcdcca54e964f75fcba6c975f5b1eb5258727f8129441c56087cfe'
 'd289a1564cb71833b8982e645bb658b75476a468476a20185fa28cd81055']


In [138]:
# Use DataFrame.column.value_counts() method
cnt = train_data.worker_name.value_counts()
print(cnt.head(50))

8fa54171acbc9f16704c03f137ac053ede9a97432cfc94e7d1d6893d9b5a    1
a5dbbf7023150fead4a600984167a0bb3daab1860f82717679ffc8c753e9    1
d481bd096165b7acbc5e82840586167f95121c45f301ee75548d228c5dc6    1
7d85f4fe79ba1c1a6834fd53968b2b3b0e14457a64c92880ba8b1e3095fb    1
3a72868cacc3a08c1c468624c058061cfa58eedb96f146f4a87ad202d07d    1
7324b2bb74da5388c038b05dd2031c61ecccb977e00b241db9ca73986ead    1
2757be3e21c7f6d4f72c302edd96000714f09113bc56a63460d2ebf6f945    1
bac8f2ee8032589b8699effcb6de64d3f4c80e5435ea7b2b17c2fd9d6eb8    1
216cf6c9f8e82604d5eb41411105af1cc14d8c083b672c71f95b3e09b75e    1
95da7c4b1d1f6afe7a6ab0782576b6b72a693192739b2c06583df210f156    1
2febdee0fb9d967b501addd976e21940843f064ea88efd8be8cce394a43f    1
e130d7869ab37503306918f990a347651a6924de00315bc4db59a5e7ec19    1
3bee0825861bf738fea59bb15ce5ce5aac92dee8a10c099ca916ca6a2ffd    1
488039fabbd4fe07cb5f3f578a9f80be949acc85d977d073cb9966ab7a68    1
e8d8ec42a56ca80c20427dab0f4e60c024cc25ee8da10df671cd8d3e808a    1
7cdb7750fe

In [223]:
drop_data = train_data.drop(train_data.columns[[0]], axis=1)
drop_data

,machine,start_time,end_time,machine_cpu_iowait,machine_cpu_kernel,machine_cpu_usr,machine_gpu,machine_load_1,machine_net_receive,machine_num_worker,machine_cpu
0,5248e29b2b7f2b5f8b974608,2134401,2134599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5248e29b2b7f2b5f8b974608,2806218,2806331,0.006080,9.226264,40.158927,192.551088,89.731273,3.845883e+08,15.672816,49.391271
2,5248e29b2b7f2b5f8b974608,3084090,3084411,0.000699,10.140853,40.807522,150.497242,72.377214,7.027994e+08,14.234469,50.949074
3,5248e29b2b7f2b5f8b974608,2119980,2134354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5248e29b2b7f2b5f8b974608,2164803,2171278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
499994,274faea7c2b83356d5de3b77,3132103,3133131,0.004562,4.102616,17.383334,51.223102,21.956170,5.512585e+08,5.043801,21.490512
499995,274faea7c2b83356d5de3b77,3211881,3211982,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290
499996,274faea7c2b83356d5de3b77,3211882,3211973,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290
499997,274faea7c2b83356d5de3b77,3244419,3244470,0.002670,1.787369,18.590296,21.676219,20.071826,5.256312e+07,5.345062,20.380335


In [224]:
drop_data.isna().sum()

machine                    0
start_time                 0
end_time                   0
machine_cpu_iowait     42111
machine_cpu_kernel     59422
machine_cpu_usr        43613
machine_gpu            53859
machine_load_1         50347
machine_net_receive    46597
machine_num_worker     52708
machine_cpu            60953
dtype: int64

In [225]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(drop_data['machine'])
drop_data['machine'] = le.transform(drop_data['machine'])

In [226]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_data['job_name'])
test_data['job_name'] = le.transform(test_data['job_name'])

In [227]:
le = preprocessing.LabelEncoder()
le.fit(test_data['task_name'])
test_data['task_name'] = le.transform(test_data['task_name'])

In [228]:
le = preprocessing.LabelEncoder()
le.fit(test_data['status'])
test_data['status'] = le.transform(test_data['status'])

In [229]:
le = preprocessing.LabelEncoder()
le.fit(test_data['gpu_type'])
test_data['gpu_type'] = le.transform(test_data['gpu_type'])

In [230]:
drop_data['machine'] = drop_data['machine'].astype('float64')

In [231]:
test_data['job_name'] = test_data['job_name'].astype('float64')
test_data['task_name'] = test_data['task_name'].astype('float64')
test_data['status'] = test_data['status'].astype('float64')
test_data['gpu_type'] = test_data['gpu_type'].astype('float64')

In [232]:
drop_data["machine_cpu_iowait"].fillna(drop_data["machine_cpu_iowait"].median(),inplace=True)
drop_data["machine_cpu_kernel"].fillna(drop_data["machine_cpu_kernel"].median(),inplace=True)
drop_data["machine_cpu_usr"].fillna(drop_data["machine_cpu_usr"].median(),inplace=True)
drop_data["machine_gpu"].fillna(drop_data["machine_gpu"].median(),inplace=True)
drop_data["machine_load_1"].fillna(drop_data["machine_load_1"].median(),inplace=True)
drop_data["machine_net_receive"].fillna(drop_data["machine_net_receive"].median(),inplace=True)
drop_data["machine_num_worker"].fillna(drop_data["machine_num_worker"].median(),inplace=True)
drop_data["machine_cpu"].fillna(drop_data["machine_cpu"].median(),inplace=True)

In [233]:
test_data["plan_cpu"].fillna(test_data["plan_cpu"].median(),inplace=True)
test_data["plan_mem"].fillna(test_data["plan_mem"].median(),inplace=True)
test_data["plan_gpu"].fillna(test_data["plan_gpu"].median(),inplace=True)
test_data["gpu_type"].fillna(test_data["gpu_type"].median(),inplace=True)

In [234]:
drop_data.dtypes

machine                float64
start_time               int64
end_time                 int64
machine_cpu_iowait     float64
machine_cpu_kernel     float64
machine_cpu_usr        float64
machine_gpu            float64
machine_load_1         float64
machine_net_receive    float64
machine_num_worker     float64
machine_cpu            float64
dtype: object

In [235]:
drop_data.isna().sum()

machine                0
start_time             0
end_time               0
machine_cpu_iowait     0
machine_cpu_kernel     0
machine_cpu_usr        0
machine_gpu            0
machine_load_1         0
machine_net_receive    0
machine_num_worker     0
machine_cpu            0
dtype: int64

In [236]:
test_data.dtypes

job_name      float64
task_name     float64
inst_num        int64
status        float64
start_time      int64
end_time        int64
plan_cpu      float64
plan_mem      float64
plan_gpu      float64
gpu_type      float64
dtype: object

In [237]:
test_data.isna().sum()

job_name      0
task_name     0
inst_num      0
status        0
start_time    0
end_time      0
plan_cpu      0
plan_mem      0
plan_gpu      0
gpu_type      0
dtype: int64

In [ ]:
#combine data, build cmb_data,cmb_target

In [238]:
cmb_data = pd.concat([drop_data, test_data], axis=1) 

In [239]:
cmb_data

,machine,start_time,end_time,machine_cpu_iowait,machine_cpu_kernel,machine_cpu_usr,machine_gpu,machine_load_1,machine_net_receive,machine_num_worker,...,job_name,task_name,inst_num,status,start_time,end_time,plan_cpu,plan_mem,plan_gpu,gpu_type
0,265.0,2134401,2134599,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,...,101299.0,14.0,1,2.0,675282,675318,600.0,29.296875,50.0,5.0
1,265.0,2806218,2806331,0.006080,9.226264,40.158927,192.551088,89.731273,3.845883e+08,15.672816,...,277436.0,14.0,1,2.0,731739,731772,600.0,29.296875,50.0,5.0
2,265.0,3084090,3084411,0.000699,10.140853,40.807522,150.497242,72.377214,7.027994e+08,14.234469,...,245339.0,14.0,2,2.0,591455,802150,200.0,4.882812,50.0,5.0
3,265.0,2119980,2134354,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,...,27468.0,14.0,1,2.0,687588,807680,100.0,0.976562,50.0,5.0
4,265.0,2164803,2171278,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,...,65793.0,14.0,1,2.0,610350,813276,100.0,0.976562,50.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,125.0,3132103,3133131,0.004562,4.102616,17.383334,51.223102,21.956170,5.512585e+08,5.043801,...,193468.0,15.0,1,2.0,4480035,4483303,600.0,29.296875,100.0,0.0
499995,125.0,3211881,3211982,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,...,162425.0,15.0,1,2.0,4482654,4483307,600.0,29.296875,100.0,1.0
499996,125.0,3211882,3211973,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,...,413210.0,15.0,1,2.0,4482651,4483320,600.0,29.296875,100.0,1.0
499997,125.0,3244419,3244470,0.002670,1.787369,18.590296,21.676219,20.071826,5.256312e+07,5.345062,...,366492.0,15.0,1,2.0,4482936,4483322,600.0,29.296875,25.0,2.0


In [240]:
cmb_data.dtypes

machine                float64
start_time               int64
end_time                 int64
machine_cpu_iowait     float64
machine_cpu_kernel     float64
machine_cpu_usr        float64
machine_gpu            float64
machine_load_1         float64
machine_net_receive    float64
machine_num_worker     float64
machine_cpu            float64
job_name               float64
task_name              float64
inst_num                 int64
status                 float64
start_time               int64
end_time                 int64
plan_cpu               float64
plan_mem               float64
plan_gpu               float64
gpu_type               float64
dtype: object

In [241]:
cmb_data.isna().sum()

machine                0
start_time             0
end_time               0
machine_cpu_iowait     0
machine_cpu_kernel     0
machine_cpu_usr        0
machine_gpu            0
machine_load_1         0
machine_net_receive    0
machine_num_worker     0
machine_cpu            0
job_name               0
task_name              0
inst_num               0
status                 0
start_time             0
end_time               0
plan_cpu               0
plan_mem               0
plan_gpu               0
gpu_type               0
dtype: int64

In [242]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(cmb_data))

In [243]:
print(normalizer.mean.numpy())

[[3.668e+02 4.402e+06 4.411e+06 4.815e-03 9.729e+00 2.685e+01 1.384e+02
  6.466e+01 4.594e+08 9.850e+00 3.638e+01 2.079e+05 1.433e+01 5.490e+00
  1.967e+00 2.712e+06 2.718e+06 6.594e+02 2.703e+01 6.555e+01 1.597e+00]]


In [244]:
first = np.array(cmb_data[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[2.65e+02 2.13e+06 2.13e+06 2.74e-03 8.22e+00 2.46e+01 1.16e+02 3.79e+01
  4.20e+08 8.96e+00 3.42e+01 1.01e+05 1.40e+01 1.00e+00 2.00e+00 6.75e+05
  6.75e+05 6.00e+02 2.93e+01 5.00e+01 5.00e+00]]

Normalized: [[-0.46 -1.87 -1.88 -0.25 -0.2  -0.17 -0.21 -0.28 -0.13 -0.16 -0.13 -0.89
  -0.13 -0.24  0.18 -2.15 -2.15 -0.13  0.15 -0.29  1.77]]


In [245]:
cmb_target = cmb_data.drop(cmb_data.columns[[17,18,19]], axis=1)
cmb_target

,machine,start_time,end_time,machine_cpu_iowait,machine_cpu_kernel,machine_cpu_usr,machine_gpu,machine_load_1,machine_net_receive,machine_num_worker,machine_cpu,job_name,task_name,inst_num,status,start_time,end_time,gpu_type
0,265.0,2134401,2134599,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,34.221865,101299.0,14.0,1,2.0,675282,675318,5.0
1,265.0,2806218,2806331,0.006080,9.226264,40.158927,192.551088,89.731273,3.845883e+08,15.672816,49.391271,277436.0,14.0,1,2.0,731739,731772,5.0
2,265.0,3084090,3084411,0.000699,10.140853,40.807522,150.497242,72.377214,7.027994e+08,14.234469,50.949074,245339.0,14.0,2,2.0,591455,802150,5.0
3,265.0,2119980,2134354,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,34.221865,27468.0,14.0,1,2.0,687588,807680,5.0
4,265.0,2164803,2171278,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,34.221865,65793.0,14.0,1,2.0,610350,813276,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,125.0,3132103,3133131,0.004562,4.102616,17.383334,51.223102,21.956170,5.512585e+08,5.043801,21.490512,193468.0,15.0,1,2.0,4480035,4483303,0.0
499995,125.0,3211881,3211982,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290,162425.0,15.0,1,2.0,4482654,4483307,1.0
499996,125.0,3211882,3211973,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290,413210.0,15.0,1,2.0,4482651,4483320,1.0
499997,125.0,3244419,3244470,0.002670,1.787369,18.590296,21.676219,20.071826,5.256312e+07,5.345062,20.380335,366492.0,15.0,1,2.0,4482936,4483322,2.0


In [246]:
cmb_data.columns.values[1] = "machine_start_time" 
cmb_data.columns.values[2] = "machine_end_time" 
cmb_data.columns.values[-3] = "job_start_time" 
cmb_data.columns.values[-2] = "job_end_time" 
print(cmb_data.columns)

Index(['machine', 'machine_start_time', 'machine_end_time',
       'machine_cpu_iowait', 'machine_cpu_kernel', 'machine_cpu_usr',
       'machine_gpu', 'machine_load_1', 'machine_net_receive',
       'machine_num_worker', 'machine_cpu', 'job_name', 'task_name',
       'inst_num', 'status', 'start_time', 'end_time', 'plan_cpu',
       'job_start_time', 'job_end_time', 'gpu_type'],
      dtype='object')


In [247]:
# Using the values attribute
cmb_target.columns.values[1] = "machine_start_time" 
cmb_target.columns.values[2] = "machine_end_time" 
cmb_target.columns.values[-3] = "job_start_time" 
cmb_target.columns.values[-2] = "job_end_time" 
print(cmb_target.columns)

Index(['machine', 'machine_start_time', 'machine_end_time',
       'machine_cpu_iowait', 'machine_cpu_kernel', 'machine_cpu_usr',
       'machine_gpu', 'machine_load_1', 'machine_net_receive',
       'machine_num_worker', 'machine_cpu', 'job_name', 'task_name',
       'inst_num', 'status', 'job_start_time', 'job_end_time', 'gpu_type'],
      dtype='object')


In [248]:
cmb_target

,machine,machine_start_time,machine_end_time,machine_cpu_iowait,machine_cpu_kernel,machine_cpu_usr,machine_gpu,machine_load_1,machine_net_receive,machine_num_worker,machine_cpu,job_name,task_name,inst_num,status,job_start_time,job_end_time,gpu_type
0,265.0,2134401,2134599,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,34.221865,101299.0,14.0,1,2.0,675282,675318,5.0
1,265.0,2806218,2806331,0.006080,9.226264,40.158927,192.551088,89.731273,3.845883e+08,15.672816,49.391271,277436.0,14.0,1,2.0,731739,731772,5.0
2,265.0,3084090,3084411,0.000699,10.140853,40.807522,150.497242,72.377214,7.027994e+08,14.234469,50.949074,245339.0,14.0,2,2.0,591455,802150,5.0
3,265.0,2119980,2134354,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,34.221865,27468.0,14.0,1,2.0,687588,807680,5.0
4,265.0,2164803,2171278,0.002745,8.215308,24.573923,115.901736,37.904131,4.198602e+08,8.964171,34.221865,65793.0,14.0,1,2.0,610350,813276,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,125.0,3132103,3133131,0.004562,4.102616,17.383334,51.223102,21.956170,5.512585e+08,5.043801,21.490512,193468.0,15.0,1,2.0,4480035,4483303,0.0
499995,125.0,3211881,3211982,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290,162425.0,15.0,1,2.0,4482654,4483307,1.0
499996,125.0,3211882,3211973,0.002716,1.676733,17.442841,29.884499,18.782778,5.283590e+07,5.098641,19.122290,413210.0,15.0,1,2.0,4482651,4483320,1.0
499997,125.0,3244419,3244470,0.002670,1.787369,18.590296,21.676219,20.071826,5.256312e+07,5.345062,20.380335,366492.0,15.0,1,2.0,4482936,4483322,2.0


In [249]:
cmb_target.isna().sum()

machine                0
machine_start_time     0
machine_end_time       0
machine_cpu_iowait     0
machine_cpu_kernel     0
machine_cpu_usr        0
machine_gpu            0
machine_load_1         0
machine_net_receive    0
machine_num_worker     0
machine_cpu            0
job_name               0
task_name              0
inst_num               0
status                 0
job_start_time         0
job_end_time           0
gpu_type               0
dtype: int64

In [172]:
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.9 MB/s eta 0:00:00


In [202]:
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
cmb_data, cmb_target = load_linnerud(return_X_y=True)
regr = MultiOutputRegressor(Ridge(random_state=123)).fit(cmb_data, cmb_target)
regr.predict(cmb_data[[3]])

array([[183.126,  35.758,  55.38 ]])

In [205]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

# get the dataset
def get_dataset():
	X, y = make_regression(n_samples=1000, n_features=18, n_informative=5, n_targets=3, random_state=2)
	return X, y

In [206]:
# get the model
def get_model(n_inputs=18, n_outputs=3):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [ ]:
#yeah samajh naahi aya

In [ ]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		print('>%.3f' % mae)
		results.append(mae)
	return results

In [209]:
# use mlp for prediction on multi-output regression
from numpy import asarray
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense

# get the dataset
cmb_data, cmb_target = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)

# get the model
def get_model(n_inputs=18, n_outputs=3):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
	model.compile(loss='mae', optimizer='adam')
	return model

In [210]:
n_inputs, n_outputs = cmb_data.shape[1],cmb_target.shape[1]
# get model
model = get_model(n_inputs, n_outputs)

In [211]:
# fit the model on all data
model.fit(cmb_data, cmb_target, verbose=0, epochs=100)

In [258]:
model.save('tal1.h5')

In [ ]:
#isko predict krny kylye values dalni hai

In [ ]:
row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
newX = asarray([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])

In [253]:
import csv

In [256]:
cmb_data.to_csv('cmb_data.csv', index=False)

In [257]:
from google.colab import files
cmb_target.to_csv('cmb_target.csv')